**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-k5t63y38
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-k5t63y38
  Resolved https://github.com/Kaggle/learntools.git to commit 3f8b29f847615177ba31b43aedefbb988c1c5fc1
  Preparing metadata (setup.py) ... - \ done
  Created wheel for learntools: filename=learntools-0.3.4-py3-none-any.whl size=268981 sha256=ffbdd9ad3135c129b495efa7c981e6cd37b8c29d34beddd8ff8bb7d89f1e8d2d
  Stored in directory: /tmp/pip-ephem-wheel-cache-mcju53mg/wheels/2f/6c/3c/aa9f50cfb5a862157cb4c7a5b34881828cf45404698255dced
Successfully built learntools
Using Kaggle's public dataset BigQuery integration.
Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
# from google.cloud import bigquery
from google.cloud import bigquery

# Create a "Client" object
# client = bigquery.Client()
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
# dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
# dataset = client.get_dataset(dataset_ref)
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
# table_ref = dataset_ref.table("taxi_trips")
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
# table = client.get_table(table_ref)
table = client.get_table(table_ref)

# Preview the first five lines of the table
# client.list_rows(table, max_results=5).to_dataframe()
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,a116ecd87f2ad76f9d68785b28800e61dbd8618b,83cdef885e81832f503b6929e7fe568508653c46268f84...,2019-04-13 00:00:00+00:00,2019-04-13 00:00:00+00:00,480,1.4,NaN,NaN,NaN,NaN,...,0.0,7.50,Cash,Taxi Affiliation Services,NaN,NaN,None,NaN,NaN,None
1,7a0a1d74891209f6bb518f436bab4f2f0c1c89f3,805d7d7ca4bbf72ee21e530f8d16935d3d72e585ca8fa3...,2019-04-13 00:00:00+00:00,2019-04-13 00:15:00+00:00,840,3.0,NaN,NaN,NaN,NaN,...,0.0,13.50,Credit Card,Choice Taxi Association,NaN,NaN,None,NaN,NaN,None
2,98dee200dc70739fa746c271cf3294aa25caa516,0ec0b176586a8986d42e5c32220e1ecd8e529d02e32925...,2019-04-13 00:00:00+00:00,2019-04-13 00:00:00+00:00,480,0.0,NaN,NaN,NaN,NaN,...,0.0,8.76,Credit Card,Star North Management LLC,NaN,NaN,None,NaN,NaN,None
3,391ecb2d3c373fd2d48eb24103c5c83a963ee733,515dbaaba624daeb95c3dfefb93bfc1764b99ed2ff96b7...,2019-04-13 00:00:00+00:00,2019-04-13 00:00:00+00:00,360,0.0,NaN,NaN,NaN,NaN,...,1.0,10.25,Credit Card,Taxi Affiliation Services,NaN,NaN,None,NaN,NaN,None
4,84afc34511d3d73230c836d865d235cb25d9da7f,4e52e33e251a85225aa90043167cc14fe93e169568e682...,2019-04-13 00:00:00+00:00,2019-04-13 00:00:00+00:00,240,0.8,NaN,1.703108e+10,NaN,8.0,...,1.0,6.25,No Charge,Star North Management LLC,NaN,NaN,None,41.899156,-87.626211,POINT (-87.6262105324 41.8991556134)


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [3]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips)
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

,trip_date,avg_num_trips
0,2016-01-30,80749.857143
1,2016-03-28,78639.428571
2,2016-02-21,89790.714286
3,2016-03-23,88152.714286
4,2016-01-09,79842.000000


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [4]:
# Lines below will give you a hint or solution code
q_1.hint()
q_1.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Use the **AVG()** function. Write an **OVER** clause with that orders the rows with the `trip_date` column and uses a window that includes the 3 preceding rows, the current row, and the following 3 rows.

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

```

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [5]:
# Amend the query below

# trip_number_query = """
#                     SELECT pickup_community_area,
#                         trip_start_timestamp,
#                         trip_end_timestamp
#                     FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
#                     WHERE DATE(trip_start_timestamp) = '2013-10-03'
#                     """

trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                            OVER (
                                  PARTITION BY pickup_community_area
                                  ORDER BY trip_start_timestamp
                                 ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                    """

trip_number_result = client.query(trip_number_query).result().to_dataframe()
# Check your answer
q_2.check()

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,24.0,2013-10-03 00:00:00+00:00,2013-10-03 00:00:00+00:00,1
1,24.0,2013-10-03 00:00:00+00:00,2013-10-03 00:00:00+00:00,1
2,24.0,2013-10-03 00:00:00+00:00,2013-10-03 00:00:00+00:00,1
3,24.0,2013-10-03 00:00:00+00:00,2013-10-03 00:00:00+00:00,1
4,24.0,2013-10-03 00:00:00+00:00,2013-10-03 00:00:00+00:00,1


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [6]:
# Lines below will give you a hint or solution code
q_2.hint()
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> You don't need a **GROUP BY** clause.  Use the **RANK()** function.  Your **OVER** clause should order the rows by the `trip_start_timestamp` column and break the data into partitions based on `pickup_community_area`.

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                            OVER (
                                  PARTITION BY pickup_community_area
                                  ORDER BY trip_start_timestamp
                                 ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                    """

trip_number_result = client.query(trip_number_query).result().to_dataframe()

```

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://storage.googleapis.com/kaggle-media/learn/images/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [7]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp, 1) 
                               OVER (
                                    PARTITION BY taxi_id 
                                    ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

# Check your answer
q_3.check()

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,001330b81e23412049f9c3eff5b6e972a91afe59c9aa36...,2013-10-03 17:45:00+00:00,2013-10-03 17:45:00+00:00,-60.0
1,0304163722b8dc5022e85cffef32025187082e7d118848...,2013-10-03 21:45:00+00:00,2013-10-03 21:30:00+00:00,165.0
2,1a9f0188874ee240e95f2713388e0ee930b07996909b35...,2013-10-03 17:45:00+00:00,2013-10-03 17:45:00+00:00,1035.0
3,1ffbf53dc521eb3df9f4ec51b9e2c75882ec5d78a6f3d6...,2013-10-03 18:15:00+00:00,2013-10-03 18:15:00+00:00,495.0
4,3183528d2ee2b082ce5ac1fa8af0f5675c9ac4958ed383...,2013-10-03 05:30:00+00:00,2013-10-03 05:30:00+00:00,255.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [8]:
# Lines below will give you a hint or solution code
q_3.hint()
q_3.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> The `TIMESTAMP_DIFF()` function takes three arguments, where the first (`trip_start_timestamp`) and the last (`MINUTE`) are provided for you.  This function provides the time difference (in minutes) of the timestamps in the first two arguments. You need only fill in the second argument, which should use the **LAG()** function to pull the timestamp corresponding to the end of the previous trip (for the same `taxi_id`).

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp, 1) OVER (PARTITION BY taxi_id ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

break_time_result = client.query(break_time_query).result().to_dataframe()

```

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*